## Language Translation Using sequence to sequence 

In [1]:
#importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
# Path to the data txt file on disk.
data_path = r'D:\data science\Projects that cannot be comitted\Language translation using Sequence to sequence from english to french\fra.txt'
number_samples = 10000       # Number of samples to train on.

In [2]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_texts

In [ ]:
target_texts

In [ ]:
input_characters


In [ ]:
len(input_characters)

In [ ]:
target_characters


In [ ]:
len(target_characters)

# sorting input character and target characters 

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [ ]:
input_characters